# Focus Filter - Intelligent Notification Filtering Agent

This notebook implements an AI agent that intelligently filters and manages notifications by:
- Classifying notifications as urgent, irrelevant, or less urgent
- Taking appropriate actions (pass through, block, or store in memory)
- Learning from patterns to improve over time

## Contest Track: Concierge
## Key Concepts Demonstrated:
1. Multi-agent system (sequential agents)
2. Custom tools for notification management
3. Sessions & Memory (long-term memory storage)
4. Observability (logging and tracing)
5. Agent evaluation (LLM-as-judge)

In [ ]:
# Install required packages
# !pip install openai python-dotenv

In [ ]:
# Imports and Setup
import os
import json
import logging
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# OpenAI API key (set via environment variable or Kaggle Secrets)
# For Kaggle: Use kaggle_secrets.UserSecretsClient().get_secret("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    try:
        from kaggle_secrets import UserSecretsClient
        OPENAI_API_KEY = UserSecretsClient().get_secret("OPENAI_API_KEY")
        os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    except:
        logger.warning("OPENAI_API_KEY not found. Please set it in your environment or Kaggle Secrets.")

print("✅ Setup complete")

In [ ]:
# Notification Data Structure
@dataclass
class Notification:
    """Represents a notification from an app or service"""
    id: str
    app: str
    title: str
    body: str
    timestamp: str
    category: Optional[str] = None
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "app": self.app,
            "title": self.title,
            "body": self.body,
            "timestamp": self.timestamp,
            "category": self.category
        }
    
    def __str__(self) -> str:
        return f"[{self.app}] {self.title}: {self.body}"

# Simple in-memory storage for demonstration
class NotificationMemory:
    """Simple memory store for less urgent notifications"""
    def __init__(self):
        self.memories: List[Dict] = []
    
    def store(self, notification: Notification, extracted_fact: str):
        """Store a notification fact in memory"""
        memory = {
            "notification_id": notification.id,
            "app": notification.app,
            "extracted_fact": extracted_fact,
            "timestamp": notification.timestamp,
            "stored_at": datetime.now().isoformat()
        }
        self.memories.append(memory)
        logger.info(f"💾 Stored memory: {extracted_fact}")
        return memory
    
    def get_all(self) -> List[Dict]:
        """Retrieve all stored memories"""
        return self.memories
    
    def search(self, query: str) -> List[Dict]:
        """Simple keyword search (would use vector DB in production)"""
        query_lower = query.lower()
        return [
            m for m in self.memories
            if query_lower in m["extracted_fact"].lower() or 
               query_lower in m["app"].lower()
        ]

# Initialize memory store
memory_store = NotificationMemory()
print("✅ Notification and Memory classes initialized")

In [ ]:
# Custom Tools for Notification Management
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

def display_urgent_notification(notification: Notification) -> str:
    """
    Display an urgent notification to the user immediately.
    This tool is called when a notification requires immediate attention.
    """
    logger.info(f"🚨 URGENT: {notification}")
    result = f"URGENT NOTIFICATION DISPLAYED:\n{notification}"
    print(f"\n{'='*60}")
    print(f"🚨 URGENT NOTIFICATION")
    print(f"{'='*60}")
    print(f"App: {notification.app}")
    print(f"Title: {notification.title}")
    print(f"Body: {notification.body}")
    print(f"{'='*60}\n")
    return result

def block_notification(notification: Notification, reason: str) -> str:
    """
    Block/suppress an irrelevant notification.
    This tool is called when a notification is determined to be noise.
    """
    logger.info(f"🚫 BLOCKED: {notification.app} - {notification.title} (Reason: {reason})")
    result = f"Notification blocked: {notification.app} - {reason}"
    print(f"🚫 Blocked: [{notification.app}] {notification.title} - {reason}")
    return result

def save_notification_memory(notification: Notification, extracted_fact: str) -> str:
    """
    Save a less urgent notification as a memory for later review.
    This tool extracts key information and stores it for future reference.
    """
    memory = memory_store.store(notification, extracted_fact)
    result = f"Memory stored: {extracted_fact}"
    print(f"💾 Saved to memory: {extracted_fact}")
    return result

print("✅ Custom tools defined")

In [ ]:
# Agent System Instructions and Classification Logic
SYSTEM_INSTRUCTIONS = """You are Focus Filter, an intelligent notification filtering agent.

Your job is to analyze notifications and classify them into one of three categories:

1. **URGENT**: Requires immediate attention or action
   - Security alerts (bank, account access)
   - Critical deadlines or time-sensitive tasks
   - Emergency communications
   - Important personal messages requiring immediate response

2. **IRRELEVANT**: Noise that should be blocked
   - Social media likes, follows, generic updates
   - Marketing/promotional content
   - Low-value informational updates
   - Spam or unwanted notifications

3. **LESS URGENT**: Important but not immediate - should be stored in memory
   - Project updates, deadline changes
   - Informational updates worth remembering
   - Non-critical but useful information
   - Things the user might want to reference later

For each notification, you must:
1. Classify it into one of the three categories
2. Provide reasoning for your classification
3. If LESS URGENT, extract the key fact/information to store

Use the available tools to take action:
- display_urgent_notification() for URGENT items
- block_notification() for IRRELEVANT items  
- save_notification_memory() for LESS URGENT items (with extracted fact)

Be conservative with URGENT - only use it for truly time-sensitive or critical items."""

def classify_notification(notification: Notification) -> Dict:
    """
    Use LLM to classify a notification and determine the appropriate action.
    This is the core reasoning step of the agent.
    """
    if not client:
        logger.error("OpenAI client not initialized. Please set OPENAI_API_KEY.")
        return {"error": "API key not configured"}
    
    # Prepare the prompt
    prompt = f"""Analyze this notification and classify it:

Notification:
App: {notification.app}
Title: {notification.title}
Body: {notification.body}
Timestamp: {notification.timestamp}

Classify this notification as URGENT, IRRELEVANT, or LESS_URGENT.

Respond in JSON format:
{{
    "classification": "URGENT" | "IRRELEVANT" | "LESS_URGENT",
    "reasoning": "brief explanation of why",
    "extracted_fact": "if LESS_URGENT, extract the key fact to remember (otherwise null)"
}}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using gpt-4o-mini for cost efficiency
            messages=[
                {"role": "system", "content": SYSTEM_INSTRUCTIONS},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"},
            temperature=0.3
        )
        
        result = json.loads(response.choices[0].message.content)
        logger.info(f"Classification: {result['classification']} - {result['reasoning']}")
        return result
        
    except Exception as e:
        logger.error(f"Error classifying notification: {e}")
        return {"error": str(e), "classification": "LESS_URGENT", "reasoning": "Error occurred, defaulting to store"}

print("✅ Classification function ready")

In [ ]:
# Main Agent Execution Function
def process_notification(notification: Notification) -> Dict:
    """
    Main agent workflow: classify notification and take appropriate action.
    This demonstrates the agentic loop: Get Mission → Think → Act → Observe
    """
    logger.info(f"📥 Processing notification: {notification.id}")
    
    # Step 1: Think - Classify the notification
    classification_result = classify_notification(notification)
    
    if "error" in classification_result:
        return {"status": "error", "details": classification_result}
    
    classification = classification_result.get("classification", "LESS_URGENT")
    reasoning = classification_result.get("reasoning", "")
    extracted_fact = classification_result.get("extracted_fact", "")
    
    # Step 2: Act - Take action based on classification
    action_result = None
    
    if classification == "URGENT":
        action_result = display_urgent_notification(notification)
        
    elif classification == "IRRELEVANT":
        action_result = block_notification(notification, reasoning)
        
    elif classification == "LESS_URGENT":
        # Extract fact if not provided
        if not extracted_fact:
            extracted_fact = f"{notification.title}: {notification.body}"
        action_result = save_notification_memory(notification, extracted_fact)
    
    # Step 3: Observe - Log the decision
    result = {
        "notification_id": notification.id,
        "classification": classification,
        "reasoning": reasoning,
        "action_taken": action_result,
        "timestamp": datetime.now().isoformat()
    }
    
    logger.info(f"✅ Processed notification {notification.id}: {classification}")
    return result

print("✅ Agent execution function ready")

In [ ]:
# Notification Simulation Tool
def simulate_notification(app: str, title: str, body: str, category: Optional[str] = None) -> Notification:
    """
    Simulate a notification from an app.
    In production, this would be replaced with actual device notification listeners.
    """
    notification = Notification(
        id=f"notif_{datetime.now().strftime('%Y%m%d%H%M%S%f')}",
        app=app,
        title=title,
        body=body,
        timestamp=datetime.now().isoformat(),
        category=category
    )
    return notification

# Test notifications
test_notifications = [
    ("Banking App", "Security Alert", "Your bank flagged suspicious activity on your account. Please verify immediately."),
    ("Social Media", "New Like", "3 new people liked your photo."),
    ("Project Manager", "Deadline Update", "Your project deadline has moved to Tuesday."),
    ("Email", "Important Meeting", "Reminder: Team meeting starts in 5 minutes."),
    ("Shopping App", "Sale Alert", "50% off on selected items! Shop now!"),
]

print("✅ Notification simulation ready")

## Running the Agent

Let's test the agent with sample notifications:

In [ ]:
# Process test notifications
results = []

print("="*70)
print("FOCUS FILTER - Processing Notifications")
print("="*70)

for app, title, body in test_notifications:
    notification = simulate_notification(app, title, body)
    result = process_notification(notification)
    results.append(result)
    print()  # Add spacing between notifications

print("="*70)
print("Processing Complete!")
print("="*70)

In [ ]:
# Display results summary
print("\n📊 Processing Summary:")
print("-" * 70)

for i, result in enumerate(results, 1):
    print(f"\n{i}. Classification: {result.get('classification', 'UNKNOWN')}")
    print(f"   Reasoning: {result.get('reasoning', 'N/A')}")

print("\n" + "="*70)
print("💾 Stored Memories:")
print("="*70)

memories = memory_store.get_all()
if memories:
    for i, memory in enumerate(memories, 1):
        print(f"\n{i}. {memory['extracted_fact']}")
        print(f"   From: {memory['app']} (stored at {memory['stored_at']})")
else:
    print("No memories stored yet.")

## Architecture Overview

This implementation demonstrates:

1. **Agent-powered by LLM**: Uses GPT-4o-mini for intelligent classification
2. **Custom Tools**: Three tools for notification management (display, block, save)
3. **Memory Management**: Simple in-memory storage (can be extended to vector DB)
4. **Observability**: Logging at each step of the agent workflow
5. **Agentic Loop**: Get Mission → Think → Act → Observe pattern

## Next Steps for Full Implementation

- Add vector database for semantic memory search
- Implement multi-agent architecture (separate agents for classification, action, memory)
- Add context engineering with few-shot examples
- Implement full observability with tracing
- Add agent evaluation framework with LLM-as-judge
- Add user preference learning